## Building A Chatbot
In this tutorial We'll go over an example of how to design and implement an LLM-powered chatbot. 

This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.


In [8]:
##Make sure you have groq API key from https://groq.com/


import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [9]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [10]:
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
response=model.invoke([HumanMessage(content="Hello , My Name is Hassan Khwileh, how you can help me todya ?"),
                       AIMessage(content="Hello Hassan, its nice to meet you, as Lead Data Scientist, "),
                       HumanMessage(content="Hey do you remember my name ?"),]),

response

(AIMessage(content="You're right! As a large language model, I don't have memory of past conversations.  \n\nSo, it's nice to meet you, Hassan Khwileh. How can I help you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 60, 'total_tokens': 112, 'completion_time': 0.094545455, 'prompt_time': 0.001625224, 'queue_time': 0.020201761999999998, 'total_time': 0.096170679}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-140d1b44-a92c-4539-96b5-6993f1f02072-0', usage_metadata={'input_tokens': 60, 'output_tokens': 52, 'total_tokens': 112}),)

In [11]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        #HumanMessage(content="Hi , My name is Hassan and I am a Lead AI Engineer"),
        #AIMessage(content="Hello Hassan Khwileh! It's nice to meet you. \n\nAs a Lead AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="As an AI, I have no memory of past conversations and no way to know your name or what you do.\n\nCan you tell me your name? 😊  I'd love to know!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 21, 'total_tokens': 64, 'completion_time': 0.078181818, 'prompt_time': 0.00014032, 'queue_time': 0.018917237, 'total_time': 0.078322138}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-386b7a5b-5a21-4acd-be37-0fb50933758b-0', usage_metadata={'input_tokens': 21, 'output_tokens': 43, 'total_tokens': 64})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Dictionary to store chat session histories.
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    Retrieve or create a chat message history for a given session ID.

    Args:
        session_id (str): The unique identifier for the chat session.

    Returns:
        BaseChatMessageHistory: The chat message history object associated with the session ID.

    Notes:
        If a session history does not exist for the provided session ID, 
        a new `ChatMessageHistory` instance is created and stored.
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
